<a href="https://colab.research.google.com/github/lucyycc/ibex-2022-05-mandarin-events-maze/blob/main/test_surpisal_for_traditional_mandarin_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

# =====================================================
# 📘 Traditional Chinese Surprisal + Frequency Notebook
# =====================================================

# Cell 1: Install dependencies
!pip install -q transformers sentencepiece huggingface_hub tqdm pandas



In [13]:
# Cell 2: Imports
import torch, math, pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM





In [14]:
# Cell 3: Load model (CKIP-GPT2 Traditional Chinese)
model_name = "ckiplab/gpt2-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
if torch.cuda.is_available():
    model.to("cuda")

In [15]:

# Cell 4: Build approximate frequency dictionary
vocab = tokenizer.get_vocab()
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])
freq_dict = {tok: 1/(rank+1) for rank, (tok, _) in enumerate(sorted_vocab)}

In [23]:
# Cell 5: Load your stimuli CSV
stimuli = pd.read_csv("/content/stimuli.csv")
print("Loaded", len(stimuli), "sentences")

Loaded 107 sentences


In [24]:
# Cell 6: Install CKIP segmenter for Traditional Chinese word segmentation
!pip install -q ckip-transformers


In [25]:
# Cell 7: Load CKIP word segmenter
from ckip_transformers.nlp import CkipWordSegmenter

In [26]:
# Cell 8: Revised surprisal function (word-level)
from torch.nn import functional as F
ws_driver = CkipWordSegmenter(model="albert-base", device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/853 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/39.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/39.8M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [33]:
# Cell 8: Revised surprisal function (word-level)
from torch.nn import functional as F

def compute_word_surprisal(sentence):
    # Segment into words (Traditional Chinese)
    words = ws_driver(sentence, use_delim=True)[0]  # returns string with spaces between words
    enc = tokenizer(words, return_tensors="pt", add_special_tokens=True)
    input_ids = enc["input_ids"]
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    with torch.no_grad():
        outputs = model(input_ids, return_dict=True)
        logits = outputs.logits
    log_probs = F.log_softmax(logits, dim=-1)

    tokens, surprisals = [], []
    for i in range(input_ids.shape[1]-1):
        target_id = input_ids[0, i+1].item()
        surprisal = -log_probs[0, i, target_id].item() / math.log(2)
        tokens.append(tokenizer.convert_ids_to_tokens(target_id))
        surprisals.append(surprisal)

    return words.split(), surprisals  # Return segmented words & surprisals

In [38]:
# Cell 7: Process all sentences
from torch.nn import functional as F
import math

def compute_word_surprisal(sentence):
    # 1️⃣ Segment the sentence (Traditional Chinese)
    words = ws_driver(sentence)[0]   # e.g., ['我', '喜歡', '吃', '蘋果']
    surprisals = []

    # 2️⃣ For each word, compute P(word | previous words)
    for i, target_word in enumerate(words):
        context = " ".join(words[:i])  # all previous words
        target = target_word

        # Encode context + target
        text = (context + " " + target).strip()
        enc = tokenizer(text, return_tensors="pt", add_special_tokens=True)
        input_ids = enc["input_ids"]
        if torch.cuda.is_available():
            input_ids = input_ids.to("cuda")

        with torch.no_grad():
            outputs = model(input_ids, return_dict=True)
            logits = outputs.logits
            log_probs = F.log_softmax(logits, dim=-1)

        # Locate target token(s)
        target_ids = tokenizer(target, return_tensors="pt", add_special_tokens=False)["input_ids"][0]
        if torch.cuda.is_available():
            target_ids = target_ids.to("cuda")

        # 3️⃣ Compute surprisal = −log P(word | context)
        # (sum subword surprisals if the word splits into multiple tokens)
        surprisal_sum = 0.0
        for j in range(len(target_ids)):
            idx = -len(target_ids) + j - 1  # positions from end
            prob = log_probs[0, idx - 1, target_ids[j]]
            surprisal_sum += -prob.item() / math.log(2)
        surprisals.append(surprisal_sum)

    return words, surprisals
